In [2]:
import json
import numpy as np
import pandas as pd
import getpass
import os
from uuid import uuid4
from guardrails.utils import retrieve_context

# Set OpenAI API key
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_community.vectorstores.utils import (
    DistanceStrategy
)
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall, answer_correctness


c:\Users\weixu\code\llm_guardrail\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load the JSON file into a Pandas DataFrame
file_path = "data/examples.json"
df = pd.read_json(file_path)

# Optionally, save the DataFrame to a CSV file
df.to_csv("data/examples.csv", index=False)


# ===============================
# Similuate the RAG Chat bot to get relevent testing data
# ===============================


# 1 Load Knowledge Base

In [9]:
with open('data/product_knowledge_base.json', 'r') as f:
    knowledge_base = json.load(f)

# Create a simple knowledge dataframe
kb_df = pd.DataFrame(knowledge_base)
kb_df.head()

,id,title,category,content
0,prod_001,Platinum Credit Card,Credit Cards,The Platinum Credit Card offers 0% interest on...
1,prod_002,First Home Buyer Loan,Home Loans,Our First Home Buyer Loan offers a 5.2% fixed ...
2,prod_003,Everyday Savings Account,Savings Accounts,The Everyday Savings Account provides a compet...
3,prod_004,Term Deposit 12 months,Term Deposits,Lock in a fixed rate of 5% p.a. for 12 months ...
4,prod_005,Smart Transaction Account,Transaction Accounts,The Smart Transaction Account offers no monthl...


# 2.2 Embed Knowledge Base

In [ ]:
# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

def score_normalizer(val: float) -> float:
    return 1 - 1 / (1 + np.exp(val))

if os.path.exists("data/faiss_index"):
    vector_store = FAISS.load_local("data/faiss_index", embeddings, allow_dangerous_deserialization=True)
else:
    # Initialize FAISS index
    index = faiss.IndexFlatIP(len(embeddings.embed_query("hello world")))
    vector_store = FAISS(
        embedding_function=embeddings,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
        relevance_score_fn = score_normalizer,
        distance_strategy=DistanceStrategy.COSINE,
    )

    # Add knowledge base embeddings to the FAISS index
    # Convert knowledge base to Document objects
    documents = [
        Document(page_content=row["content"], metadata={"source": row["title"]})
        for _, row in kb_df.iterrows()
    ]

    # Generate unique IDs for each document
    uuids = [str(uuid4()) for _ in range(len(documents))]

    # Add documents to the FAISS vector store
    vector_store.add_documents(documents=documents, ids=uuids)

    vector_store.save_local("data/faiss_index")

# 3 Similate chatbot conversation

In [ ]:
streamlit run chatbot.py

# 4 Collect Chatbot Responses

In [ ]:
import json
# Load the chat history from the JSON file
with open("data/chat_history.json", "r", encoding="utf-8") as file:
    chat_history = json.load(file)

# Convert the chat history to the desired format
examples = []
for i in range(0, len(chat_history) - 1, 2):
    if chat_history[i]["role"] == "user" and chat_history[i + 1]["role"] == "assistant":
        examples.append({
            "query": chat_history[i]["content"],
            "response": chat_history[i + 1]["content"]
        })

# Print or save the result
print(examples)

# Optionally, save the result to a new JSON file
with open("data/relevent_examples.json", "w", encoding="utf-8") as output_file:
    json.dump(examples, output_file, indent=4, ensure_ascii=False)

### manually create the reference data for each questions or use AI to pre-generate the reference data, and then human to verify the answer.

In [5]:
# examples_reference.json is generated for evaluation
# read the examples_reference.json file and convert it to the format required by the evaluation function
with open("data/relevant_examples_reference.json", "r", encoding="utf-8") as file:
    examples = json.load(file)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = FAISS.load_local("data/faiss_index", embeddings, allow_dangerous_deserialization=True)


In [6]:
# Collect evaluation data
records = []
for ex in examples_reference:
    retrieved_contexts, retrieved_contexts_string = retrieve_context(ex["query"], vector_store=vector_store, top_k=3)
    # convert retrieved_contexts to a list of strings with content
    retrieved_contexts_list = [context['content'] for context in retrieved_contexts]
    similarity = [context['score'] for context in retrieved_contexts]
    # average similarity score
    similarity = np.mean(similarity) if similarity else 0.0
    record = {
        "user_input": ex["query"],
        "response": ex["response"],
        "retrieved_contexts": retrieved_contexts_list,
        "reference": ex["reference"],
        "similarity": similarity
    }
    records.append(record)

evaluation_df = pd.DataFrame(records)
# save the evaluation data to a CSV file
evaluation_df.to_csv("data/evaluation_data.csv", index=False, encoding="utf-8-sig")
